BDLE 2023

date du document  :  18/09/2023

# TP1 2023 Préparation de données  (ETUDIANT)


Préparation de données


## Préparation du TP

Vérifier que des ressources de calcul sont allouées à votre notebook  (cf RAM  de disque indiqués en haut à droite) . Sinon cliquer sur le bouton connecter pour obtenir des ressources.




Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/ens/bdle/TP1

In [1]:
# import os
# from google.colab import drive
# drive.mount("/content/drive")

# drive_dir = "/content/drive/MyDrive/ens/bdle/TP1"
# os.makedirs(drive_dir, exist_ok=True)
# os.listdir(drive_dir)

Installer pyspark et findspark (durée 1'):


In [2]:
!pip install -q pyspark
!pip install -q findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Démarrer la session spark

In [3]:
import os
import glob

pyspark_dir = glob.glob('/usr/local/lib/python*/dist-packages/pyspark')[0]
print("pyspark directory is", pyspark_dir)
os.environ["SPARK_HOME"] = pyspark_dir
os.environ["JAVA_HOME"] = "/usr"

pyspark directory is /usr/local/lib/python3.10/dist-packages/pyspark


In [4]:
# Principaux import
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf

# pour les dataframe et udf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")

  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")

  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1695651607778


In [5]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

Nombre de partitions utilisées :  8


In [6]:
# Optionnel :
# pour l'accès à spark UI : voir https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !curl -s http://localhost:4040/api/tunnels

Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [7]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas()


Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [8]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + "\n"
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    derniere = None
    est_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          derniere = spark.sql(r)
          est_requete = ( r.lower().startswith('select')or r.lower().startswith('with'))
    if(est_requete):
      return display(derniere)
    else:
      return print('ok')

In [9]:
# facultatif (à ne pas utiliser)
# %load_ext google.colab.data_table
# %unload_ext google.colab.data_table

## Accès aux données

### URL pour l'accès aux datasets

In [10]:
# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4"
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL du dossier contenant les datasets ", PUBLIC_DATASET_URL)

URL du dossier contenant les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


### Données de mobilité

Données issues du dataset YFCC

In [11]:
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

In [12]:
from urllib import request

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

# user visits
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15"
download_file(web_dir, local_dir, "userVisits-Toro.csv")

# poi
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15"
download_file(web_dir, local_dir, "POI-Toro.csv")



os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15/userVisits-Toro.csv save in : /local/data/userVisits-Toro.csv
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15/POI-Toro.csv save in : /local/data/POI-Toro.csv


['userVisits-Toro.csv', 'POI-Toro.csv']

### Les visites

Lire les 2 premières lignes du  fichier csv en python.
Est ce que le fichier a une ligne d'entête ?
Quel caractère délimite deux valeurs consécutives dans une ligne de données ?

In [13]:
f = open(local_dir + "/" + "userVisits-Toro.csv", "r")
print(f.readline()); print(f.readline())

"photoID";"userID";"dateTaken";"poiID";"poiTheme";"poiFreq";"seqID"

7941504100;"10007579@N00";1346844688;30;"Structure";1538;1



Lire le fichier des visites *sans* préciser le type des attributs.
Par défaut, tous les attributs sont considérés comme étant de type string.

In [14]:
user_visits = spark.read.option("header", "True").option("delimiter", ";").format("csv").load(local_dir + "/" + "userVisits-Toro.csv")
user_visits.show(3)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID| dateTaken|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 3 rows

root
 |-- photoID: string (nullable = true)
 |-- userID: string (nullable = true)
 |-- dateTaken: string (nullable = true)
 |-- poiID: string (nullable = true)
 |-- poiTheme: string (nullable = true)
 |-- poiFreq: string (nullable = true)
 |-- seqID: string (nullable = true)



Lire le fichier en précisant le schéma : nom et type des attributs

In [15]:
schema = "photoID long, userID String, date Long, poiID int, poiTheme String, poiFreq int, seqID int"

user_visits = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "userVisits-Toro.csv", schema = schema)
user_visits.persist()
user_visits.createOrReplaceTempView("user_visits")
user_visits.show(10)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID|      date|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
|4885404441|10012675@N05|1142732373|    6| Cultural|    986|    2|
|4886008334|10012675@N05|1142732445|    6| Cultural|    986|    2|
|4886009150|10012675@N05|1142916492|    6| Cultural|    986|    3|
|7054481539|10012675@N05|1319327174|   13| Cultural|    964|    4|
|6908387594|10012675@N05|1319328255|   13| Cultural|    964|    4|
|6908381912|10012675@N05|1319331463|   13| Cultural|    964|    4|
|6908398496|10012675@N05|1319331886|   13| Cultural|    964|    4|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 10 rows

root
 |-- photoID: long (nullable = 

Vérifier qu'il n'y a pas de date nulle

In [16]:
%%sql
select count(1) as nb_dates_nulles
from user_visits
where date is null

-- autre solution avec "case"
--select count(case when isnull(date) then date end) as nb_date_nulles
--from user_visits

,nb_dates_nulles
0,0



Vérifier qu'il n'y a aucune séquence associée à plusieurs utilisateurs

In [17]:
%%sql
select seqID
from user_visits
group by seqID
having count(distinct userID) > 1

,seqID



#### POI_sequence

La table **POI_sequence**(seqID, poiID). Les POI visités durant une séquence.
Pour simplifier on compte chaque POI une seule fois par séquence et on ne considère pas l'ordre de visite des POI.

Rmq: il n'est pas nécessaire de préciser le userID pour identifier une séquence.

In [18]:
%%sql
create or replace temp view POI_sequence as
select distinct seqID, poiID
from user_visits;

select * from POI_sequence

,seqID,poiID
0,9,24
1,16,24
2,26,22
3,29,25
4,54,25
...,...,...
95,624,23
96,634,22
97,638,4
98,641,30


### Les lieux visités : POI
Ils sont appelés *Point Of Interest*

In [19]:
poi_schema = "poiID long, poiName String, latitude double, longitude double, theme String"

poi = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "POI-Toro.csv", schema = poi_schema)
poi.show(3)
poi.printSchema()
poi.createOrReplaceTempView("POI")

+-----+------------------+--------+---------+-----+
|poiID|           poiName|latitude|longitude|theme|
+-----+------------------+--------+---------+-----+
|    1| Air_Canada_Centre|43.64333|-79.37917|Sport|
|    2|         BMO_Field|43.63278|-79.41861|Sport|
|    3|Maple_Leaf_Gardens|43.66222|-79.38028|Sport|
+-----+------------------+--------+---------+-----+
only showing top 3 rows

root
 |-- poiID: long (nullable = true)
 |-- poiName: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- theme: string (nullable = true)



In [20]:
%%sql
cache table POI;

SELECT *
FROM POI

,poiID,poiName,latitude,longitude,theme
0,1,Air_Canada_Centre,43.643330,-79.379170,Sport
1,2,BMO_Field,43.632780,-79.418610,Sport
2,3,Maple_Leaf_Gardens,43.662220,-79.380280,Sport
3,4,Rogers_Centre,43.641390,-79.389170,Sport
4,5,Woodbine_Racetrack,43.712525,-79.602042,Sport
5,6,Art_Gallery_of_Ontario,43.653890,-79.392780,Cultural
6,7,Hockey_Hall_of_Fame,43.646976,-79.377253,Cultural
7,8,Ripley%27s_Aquarium_of_Canada,43.642481,-79.386050,Cultural
8,9,Ontario_Science_Centre,43.716670,-79.338330,Cultural
9,10,Riverdale_Farm,43.667111,-79.361294,Cultural


la liste des thèmes

In [21]:
%%sql
create or replace temp view themes as
select distinct theme
from POI
order by theme;

cache table themes;

select *
from themes

,theme
0,Amusement
1,Beach
2,Cultural
3,Shopping
4,Sport
5,Structure


## Exercice 1

#### 1) Les 10 POI les plus photographiés
indications: on peut compléter une requête SQL avec *LIMIT 10* pour borner la taille résultat (partiel) souhaité.

In [22]:
%%sql
select  poiID, count(*) as nombre
from user_visits
group by poiID
order by count(*) desc
limit 10

,poiID,nombre
0,11,4139
1,22,3603
2,21,3591
3,16,3553
4,1,3506
5,4,3056
6,7,2053
7,23,1866
8,8,1736
9,25,1701


#### 2) Les visites avec date détaillée
Définir la table Visite_date(userID, seqID, poiID, date, annee, mois, jour, heure) contenant les visites avec la date détaillée composée des attributs : année, mois, jour, heure.
Indication : la date est initialement au format "unix". Voir la fonction de conversion de date from_unixtime et la fonction extract.

In [23]:
%%sql
create or replace temp view Visite_Date as
select userID, seqID, poiID, extract(YEAR FROM FROM_UNIXTIME(date)) as year,
extract(MONTH FROM FROM_UNIXTIME(date)) as month,
extract(DAY FROM FROM_UNIXTIME(date)) as day,
extract(HOUR FROM FROM_UNIXTIME(date)) as hour
from user_visits
;
select * from Visite_Date



,userID,seqID,poiID,year,month,day,hour
0,10007579@N00,1,30,2012,9,5,11
1,10012675@N05,2,6,2006,3,19,1
2,10012675@N05,2,6,2006,3,19,1
3,10012675@N05,2,6,2006,3,19,1
4,10012675@N05,2,6,2006,3,19,1
...,...,...,...,...,...,...,...
95,10014440@N06,10,25,2008,7,9,8
96,10014440@N06,10,25,2008,7,9,8
97,10014440@N06,10,25,2008,7,9,8
98,10014440@N06,10,25,2008,7,9,8


#### 3a) Le nombre de POI par utilisateur
Le résultat (userID, nbPOI) est trié par nombre décroissant de POI.

In [24]:
%%sql
select userID, count(*) as nbPOI
from user_visits
group by userID
order by count(*) desc

,userID,nbPOI
0,48784629@N00,3265
1,39460517@N03,1730
2,7616875@N04,1665
3,84987970@N00,1430
4,86822950@N03,1403
...,...,...
95,91008793@N00,68
96,70307237@N00,67
97,13833167@N00,66
98,27914970@N04,65


#### 3b) Le nombre de POI par séquence
Définir la table Seq3plus(seqID, nbPoi) correspondant aux séquences qui contiennent au moins 3 POI distincts.
Afficher le résultat trié par nombre de POI décroissant puis numéro de séquence croissant.


In [25]:
%%sql
create or replace temp view Seq3plus as
select seqID, count(*) as nbPOI
from user_visits
group by seqID
order by count(*) desc, seqID
;


select * from seq3plus
order by nbPOI desc, seqID;

,seqID,nbPOI
0,3935,643
1,4635,412
2,3933,373
3,3011,320
4,4632,316
...,...,...
95,1305,66
96,5294,66
97,558,65
98,2474,65


#### 4) Trajectoire



a) Définir une fonction qui prend en entrée une liste de couples (date, poi) et qui retourne une liste PoI triés par date croissante. Le resultat ne contient pas 2 PoI consécutifs identiques.

In [26]:
def trierPOI(liste):
  liste.sort(key=lambda a:a[0])
  result = [liste[0][1]]
  prec = liste[0][1]
  for date,poi in liste[1:]:
    if poi != prec :
      result.append(poi)
    prec = poi
  return result

spark.udf.register("trierPOI", trierPOI, ArrayType(IntegerType()))

# # test local
print(trierPOI([(13,0),(10,2), (12,1), (14,2),(11,2)]))

[2, 1, 0, 2]


Définir la vue Trajectoire(userID, seqID, listPOI).

listPOI est la liste des POI visités, pendant la séquence seqID, dans l'ordre chronologique

Indications:
pour la sequence 687 la listPOI est [7,16,4,8,4,16]

In [27]:
%%sql
create or replace temp view Trajectoire as
  select userID, seqID, trierPOI(collect_list((date, poiID))) as listePOI
  from user_visits
  group by userID,seqID;

cache table Trajectoire;


select * from Trajectoire
where size(listePOI) =6
order by seqID


,userID,seqID,listePOI
0,14617133@N05,384,"[22, 28, 22, 28, 22, 28]"
1,20456447@N03,687,"[7, 16, 4, 8, 4, 16]"
2,20741443@N00,729,"[23, 21, 22, 7, 28, 23]"
3,27168489@N00,1358,"[16, 8, 28, 22, 23, 21]"
4,29352917@N00,1524,"[29, 30, 16, 29, 16, 29]"
5,30624156@N00,1598,"[28, 7, 30, 28, 22, 27]"
6,32827327@N03,1786,"[20, 6, 25, 11, 30, 24]"
7,33547369@N00,1877,"[23, 21, 23, 22, 23, 22]"
8,34128007@N04,1912,"[8, 16, 4, 1, 29, 16]"
9,34314322@N00,1993,"[30, 22, 28, 29, 30, 16]"


#### 5) Transitions
5a) Définir la table Transitions(poi1, poi2, nbTrans) avec nbTrans étant le nombre de déplacements directs de poi1 à poi2.
Indication: deux POI apparaissant successivement (poi1 suivi de poi2) dans une séquence forment un déplacement direct entre poi1 et poi2.
Une séquence contenant un seul POI ne correspond à aucun déplacement.

In [82]:
def transition(listePOI):

    occurrences = {}
    prec = listePOI[0]
    for poi in listePOI[1:]:
        if(prec != poi):
          key = (prec, poi)
          if key not in occurrences:
              occurrences[key] = 1
          else:
              occurrences[key] += 1
          prec = poi
    return [(k[0], k[1], v) for k, v in occurrences.items()]


spark.udf.register("transition", transition, ArrayType(StructType([
    StructField("poi1", IntegerType()),
    StructField("poi2", IntegerType()),
    StructField("nbTrans", IntegerType())
])))


transition([1,2,3,4,1,2,2,2])

[(1, 2, 2), (2, 3, 1), (3, 4, 1), (4, 1, 1)]

In [81]:
%%sql



create or replace temp view Transition as
select
  seqID,
  transition.poi1 as poi1,
  transition.poi2 as poi2,
  transition.nbTrans as nbTrans
from
  (select seqID, explode(transition(collect_list(poiID))) as transition from user_visits group by seqID) ;


select *
from Transition


,seqID,poi1,poi2,nbTrans
0,8,23,24,1
1,33,21,25,1
2,58,7,11,1
3,58,11,27,1
4,59,11,27,1
...,...,...,...,...
95,315,8,16,1
96,316,21,27,1
97,316,27,11,2
98,316,11,27,1


b) Definir la vue
Transition_relative avec des valeurs de transition relatives au nombre total de transition partant d'un POI.

Indication: créer une vue donnant le nombre total de transitions par poi.

In [ ]:
%%sql
create or replace temp view Total_transition as
select poi1, sum(nbTrans) as nbTrans
from Transition
group by poi1
;

select * from Total_transition


,poi1,nbTrans
0,13,964
1,12,481
2,18,26
3,14,141
4,6,982
5,23,1866
6,17,808
7,16,3553
8,9,278
9,10,346


In [35]:
# %%sql
# create or replace temp view Transition_relative as


### 6) DuréeVisitePOI
La durée moyenne de visite d'un POI.
Indication, pour une série d'événements consécutifs associés à un même POI dans une séquence, la durée de visite est la différence de date entre le 1er et le dernier événement.

In [55]:
import numpy as np

def dureeVisite(liste_couples):
    result = []

    dico = {}
    for poi,date in liste_couples:

        if poi in dico:
            dico[poi].append(date)
        else:
            dico[poi] = [date]

    for key, dates in dico.items():
        #min_timestamp = np.min(date) ne fonctionne pas
        #max_timestamp = np.max(date)


        min_timestamp = dates[0]
        max_timestamp = dates[0]
        for date in dates:
          if date < min_timestamp:
              min_timestamp = date
          elif date > max_timestamp:
              max_timestamp = date


        duree = max_timestamp - min_timestamp
        result.append((key, duree))



    return result

spark.udf.register("dureeVisite", dureeVisite, ArrayType(StructType([
    StructField("poi", IntegerType()),
    StructField("duree", IntegerType()),
    ] )))

# # test local
dureeVisite([(102,8), (102,11), (106,14) ,(101,15), (101,17), (102,20), (102,21), (102,24)])

[(102, 16), (106, 0), (101, 2)]

In [72]:
%%sql

create or replace temp view DureeVisitePOI as
select userID, seqID, duree_poi.poi as poi, duree_poi.duree as duree
from(select userID, seqID, explode(dureeVisite(collect_list((poiID,date)))) as duree_poi
from user_visits group by userID, seqID );


select *
from DureeVisitePOI d
order by userID, seqID

,userID,seqID,poi,duree
0,10007579@N00,1,30,0
1,10012675@N05,2,6,597
2,10012675@N05,3,6,0
3,10012675@N05,4,13,5674
4,10014440@N06,5,24,257
...,...,...,...,...
95,10575698@N02,85,28,0
96,10575698@N02,86,27,0
97,10575698@N02,87,28,0
98,10575698@N02,88,24,0


In [101]:
%%sql
create or replace temp view DureeMoyenneVisitePOI as
select poi, avg(duree) as duree_moyenne
from DureeVisitePOI
group by poi
;

select * from DureeMoyenneVisitePOI

,poi,duree_moyenne
0,13,2124.133333
1,12,3136.986486
2,18,1037.400000
3,14,965.973684
4,6,1379.530435
5,23,1468.289225
6,17,6392.653846
7,16,2971.145068
8,9,3673.694444
9,10,2013.134146


### 7) Durée moyenne de déplacement entre deux POIs.

Définir la vue **DureeDeplacement** (poi1, poi2, duree) calculant la duree moyenne de déplacement entre deux POI consécutifs dans une séquence.



In [119]:
def moy(liste):

  cpt = 0
  sum =0
  for l in liste:
    cpt+=1
    sum+=l
  return sum/cpt


In [132]:


def duree_deplacement(couple_POI):
    occurences = {}

    prec = couple_POI[0][0]
    duree_prec = couple_POI[0][1]
    for poi,duree  in couple_POI[1:]:
        if(prec != poi):
          key = (prec, poi)
          if key not in occurences:
              occurences[key] = [duree - duree_prec]
          else:
              occurences[key].append(duree - duree_prec)
          prec = poi
          duree_prec = duree


    if len(occurences) == 0
    return [(k[0], k[1], moy(v)) for k, v in occurences.items()]


spark.udf.register("duree_deplacement",duree_deplacement, ArrayType(StructType([
    StructField("poi1", IntegerType()),
    StructField("poi2", IntegerType()),
    StructField("duree", FloatType())
])))


duree_deplacement([(1,50), (2,60), (1,70), (2, 100), (3,110)])

[(1, 2, 20.0), (2, 1, 10.0), (2, 3, 10.0)]

In [140]:
%%sql
create or replace temp view DureeDeplacement as
select
  seqID,
  tuple.poi1 as poi1,
  tuple.poi2 as poi2,
  tuple.duree as duree
from
(
select seqID, explode(duree_deplacement(collect_list((poiID,date)))) as tuple
from user_visits
group by seqID);

select * from DureeDeplacement

,seqID,poi1,poi2,duree
0,8,23,24,4586.0
1,33,21,25,2857.0
2,58,7,11,16892.0
3,58,11,27,1416.0
4,59,11,27,574.0
...,...,...,...,...
95,315,8,16,1034.0
96,316,21,27,1128.0
97,316,27,11,139.0
98,316,11,27,48.0


## Exercice 2

Charger les données de Geonames

In [141]:
import zipfile

#geonames
web_dir = PUBLIC_DATASET + "geonames_ALL"

download_file(web_dir, local_dir, "allCountries.zip")

#unzip
local_file = local_dir + "/" + "allCountries.txt"
if(os.path.isfile(local_file)):
    print(local_file, "is already stored")
else:
  with zipfile.ZipFile(local_dir + "/" + "allCountries.zip", 'r') as zip_ref:
    zip_ref.extractall(local_dir)

os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=geonames_ALL/allCountries.zip save in : /local/data/allCountries.zip


['allCountries.zip', 'userVisits-Toro.csv', 'allCountries.txt', 'POI-Toro.csv']

In [142]:
geonames =  spark.read.option("header", "False").option("delimiter", "\t").format("csv").load(local_dir + "/" + "allCountries.txt")
geonames.createOrReplaceTempView("geonames")
geonames.show(3)

+-------+--------------------+--------------------+--------------------+--------+-------+---+---+---+-----+----+----+----+----+----+----+----+--------------+----------+
|    _c0|                 _c1|                 _c2|                 _c3|     _c4|    _c5|_c6|_c7|_c8|  _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|          _c17|      _c18|
+-------+--------------------+--------------------+--------------------+--------+-------+---+---+---+-----+----+----+----+----+----+----+----+--------------+----------+
|2986043|  Pic de Font Blanca|  Pic de Font Blanca|Pic de Font Blanc...|42.64991|1.53335|  T| PK| AD| null|  00|null|null|null|   0|null|2860|Europe/Andorra|2014-11-05|
|2994701|            Roc Mélé|            Roc Mele|Roc Mele,Roc Mele...|42.58765|1.74028|  T| MT| AD|AD,FR|  00|null|null|null|   0|null|2803|Europe/Andorra|2014-11-05|
|3007683|Pic des Langounelles|Pic des Langounelles|Pic des Langounelles|42.61203|1.47364|  T| PK| AD|AD,FR|  00|null|null|null|   0|null|2685|Europe/Andorr

### 1) Geonames
Définir le schéma, limité au 9 premiers attributs donnant des informations sur l'identifiant d'un POI, ses noms, sa position GPS, le code du pays..., ), d'après le document [readme.txt](https://nuage.lip6.fr/s/kaBX3sF3YEzMZHK) dans le dossier [geonames_ALL](https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4?path=%2Fgeonames_ALL)

#####1a) schéma de Geonames

Définir Geonames2 en précisant le schéma : nom et type des attributs.
Se limiter au 9 premiers attributs donnant des informations sur l'identifiant d'un POI, ses noms, sa position GPS, le code du pays..., ), d'après le document [readme.txt](https://nuage.lip6.fr/s/kaBX3sF3YEzMZHK) dans le dossier [geonames_ALL](https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4?path=%2Fgeonames_ALL)

In [143]:
%%sql
create or replace temp view Geonames2 as
select cast(_c0 as long) as geonameID,
cast(_c1 as string) as nom,
cast(_c2 as string) as asciiname,
cast(_c3 as string) as alternatenames,
cast(_c4 as float) as latitude,
cast(_c5 as float) as longitude,
cast(_c6 as string) as feature_class,
cast(_c7 as string) as feature_code,
cast(_c8 as string) as country_code
from Geonames;

select * from Geonames2

,geonameID,nom,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code
0,2986043,Pic de Font Blanca,Pic de Font Blanca,"Pic de Font Blanca,Pic du Port",42.649910,1.53335,T,PK,AD
1,2994701,Roc Mélé,Roc Mele,"Roc Mele,Roc Meler,Roc Mélé",42.587650,1.74028,T,MT,AD
2,3007683,Pic des Langounelles,Pic des Langounelles,Pic des Langounelles,42.612030,1.47364,T,PK,AD
3,3017832,Pic de les Abelletes,Pic de les Abelletes,"Pic de la Font-Negre,Pic de la Font-Nègre,Pic ...",42.525349,1.73343,T,PK,AD
4,3017833,Estany de les Abelletes,Estany de les Abelletes,"Estany de les Abelletes,Etang de Font-Negre,Ét...",42.529148,1.73362,H,LK,AD
...,...,...,...,...,...,...,...,...,...
95,3038902,Tosquers,Tosquers,None,42.565868,1.48763,T,SLP,AD
96,3038903,Canal del Tosquer,Canal del Tosquer,None,42.571140,1.52067,H,STM,AD
97,3038904,Bosc del Tosquer,Bosc del Tosquer,None,42.571892,1.52369,V,FRST,AD
98,3038905,Bosc del Tosquer,Bosc del Tosquer,None,42.537930,1.60070,V,FRST,AD


##### 1b) Extrait pour le Canada

Définir la vue Geonames_canada pour les POI situés au Canada.

In [ ]:
%%sql
create or replace temp view Geonames_canada as
select *
from Geonames2
where country_code = 'CA'

;

cache table Geonames_canada;

select * from  Geonames_canada;

,geonameID,nom,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code
0,3424953,Virgin Rocks,Virgin Rocks,None,46.428860,-50.819950,U,RFU,CA
1,3425041,Eastern Shoals,Eastern Shoals,None,46.447430,-50.485771,U,SHSU,CA
2,3426061,Downing Basin,Downing Basin,None,47.052158,-50.791630,U,TRGU,CA
3,3831549,Kane Basin,Kane Basin,"Bassin Kane,Kane Basin",79.388603,-70.425407,H,STRT,CA
4,3831631,Georges Shoal,Georges Shoal,"Georges Bank,St. George's Bank,St. George's Shoal",41.654442,-67.732422,U,SHLU,CA
...,...,...,...,...,...,...,...,...,...
95,5881729,Aalders Landing,Aalders Landing,None,44.833450,-64.915489,L,AREA,CA
96,5881730,Aalders Lang Brook,Aalders Lang Brook,None,44.866852,-64.298752,H,STM,CA
97,5881731,Aalders Lang Meadow,Aalders Lang Meadow,None,44.850151,-64.315453,T,PLN,CA
98,5881732,Aaltanhash Inlet,Aaltanhash Inlet,Aaltenash Bight,53.128540,-128.517883,H,INLT,CA


### 2) Association entre les POI et Geonames
Compléter les POI avec des attributs de geonames (par exemple name et [feature code](http://www.geonames.org/export/codes.html))

Décrire la solution que vous proposez.

In [144]:
%%sql
select p.poiID,	p.poiName,	p.latitude, p.longitude, p.theme, g.nom, g.feature_code
from POI p , Geonames2 g
where p.latitude=g.latitude and p.longitude=g.longitude

,poiID,poiName,latitude,longitude,theme,nom,feature_code
